In [348]:
import pandas as pd
import numpy as np
import cairo
import math
import ast

In [349]:
def get_sister_edge(edge: pd.Series, edgelist: pd.DataFrame) -> pd.DataFrame:
    """Return sister edges sharing the same source but different target."""
    source = edge['source']
    target = edge['target']
    return edgelist[(edgelist['source'] == source) & (edgelist['target'] != target)]


def get_parent_edges(edge: pd.Series, edgelist: pd.DataFrame) -> pd.DataFrame:
    """Return edges whose target is the source of the current edge."""
    source = edge['source']
    return edgelist[edgelist['target'] == source]

In [350]:
#render parameters
log_reduction = 3

color_map = {
    "scan": "grey",
    "tran1" : 'lightblue',
    "tran2" : "blue",
    "tran3" : 'darkblue',
    "init" : "green",
    "drop" : "purple",
    'rein': 'orange',
    'shift': 'red',
    'load': 'purple'
}

COLOURS = {
    "grey":   (0.6, 0.6, 0.6),
    "green":  (0.2, 0.7, 0.3),
    "blue":   (0.2, 0.4, 0.8),
    "purple": (0.6, 0.3, 0.7),
    'orange': (1, 0.6, 0 ),
    'darkblue': (0.05,0.05,0.6),
    'lightblue': (0.4,0.5,0.9),
    'red': (0.9,0.1,0.1)
}

In [351]:
edgelist = pd.read_csv('graph.csv', sep = '\t')

print(edgelist)

     source    target  weight  state  flux
0  (-1, -1)    (1, 0)     1.0   load   1.0
1    (3, 0)    (3, 3)     0.5   init   0.5
2   (68, 2)   (68, 0)     0.2   rein   0.0
3    (1, 0)    (3, 0)     1.0   scan   1.0
4    (3, 3)   (10, 3)     1.0  tran3   0.5
5   (26, 2)   (68, 2)     1.0  tran2   0.0
6    (3, 0)   (10, 0)     0.5   scan   0.5
7   (10, 0)  (-1, -1)     1.0   drop   0.5
8   (10, 3)  (-1, -1)     1.0   drop   0.5
9   (68, 2)  (-1, -1)     0.4   drop   0.0


In [352]:

def add_vert_category(edgelist: pd.DataFrame, log_reduction: float = 1.5, height_scale: float = 2) -> pd.DataFrame:
    """
    Add vertical direction and log-scaled x positions to the edge list.

    Args:
        edgelist: DataFrame with 'source' and 'target' as stringified tuples.
        log_reduction: Base for log scaling gaps in x positions.

    Returns:
        DataFrame with updated 'source_x', 'source_y', 'target_x', 'target_y', 'vert'.
    """

    out = edgelist.copy()

    # --- Parse source/target coordinates ---
    out['source'] = out['source'].apply(ast.literal_eval)
    out['target'] = out['target'].apply(ast.literal_eval)

    # --- Split into x/y columns ---
    for point in ['source', 'target']:
        out[f'{point}_x'] = out[point].apply(lambda v: int(v[0]))
        out[f'{point}_y'] = out[point].apply(lambda v: int(v[1]))


    # --- Adjust negative target_y values ---
    mask_neg = out['source_y'] < 0
    out.loc[mask_neg, 'source_y'] = out.loc[mask_neg, 'target_y']

    # --- Adjust negative target_y values ---
    mask_neg = out['source_x'] < 0
    out.loc[mask_neg, 'source_x'] = out.loc[mask_neg, 'target_x'] - 1

    # --- Adjust negative target_y values ---
    mask_neg = out['target_x'] < 0
    out.loc[mask_neg, 'target_x'] = out.loc[mask_neg, 'source_x']

    if log_reduction != 1: #SCALE GRAPH BASED ON LOG VALUES OF DISTANCES
        # --- Compute unique sorted x positions ---
        x_pos = sorted(set(list(out['source_x']) + [out['target_x'].iloc[-1]]))

        # --- Compute gaps and log-scale ---
        gaps = [x_pos[i+1] - x_pos[i] for i in range(len(x_pos)-1)]
        log_gaps = [math.log(g, log_reduction) + 1 for g in gaps]

        # --- Map original x to log-scaled x ---
        log_x_pos = [x_pos[0]]
        for gap in log_gaps:
            log_x_pos.append(log_x_pos[-1] + gap)


        log_map = dict(zip(x_pos, log_x_pos))

        
        out['source_x'] = out['source_x'].map(log_map)
        out['target_x'] = out['target_x'].map(log_map)



    # --- Compute vertical direction (+1/-1), 0 if same ---
    delta_y = out['target_y'] - out['source_y']
    out['vert'] = (delta_y / delta_y.abs()).fillna(0).astype(int)

    negatives = out[out['target'] == (-1,-1)]
    for idx, edge in negatives.iterrows():
        sibling = get_sister_edge(edge, edgelist=out)
        if sibling.empty:
            out.loc[idx, 'target_y'] = out.loc[idx, 'source_y'] + 0.5
        else:
            if sibling['vert'].values[0] == 1:
                out.loc[idx, 'target_y'] = out.loc[idx, 'source_y'] - 0.5
                continue
            out.loc[idx, 'target_y'] = out.loc[idx, 'source_y'] + 0.5
        
        # --- Compute vertical direction (+1/-1), 0 if same ---
    delta_y = out['target_y'] - out['source_y']
    out['vert'] = (delta_y / delta_y.abs()).fillna(0).astype(int)
    
    # --- Scale y coordinates ---
    out['source_y'] *= height_scale
    out['target_y'] *= height_scale

    out = out.sort_values(by='source_x', axis=0)

    return out

vertified = add_vert_category(edgelist=edgelist, log_reduction=log_reduction, height_scale=2)
print(vertified)


     source    target  weight  state  flux   source_x  source_y   target_x  \
0  (-1, -1)    (1, 0)     1.0   load   1.0   0.000000         0   1.000000   
3    (1, 0)    (3, 0)     1.0   scan   1.0   1.000000         0   2.630930   
1    (3, 0)    (3, 3)     0.5   init   0.5   2.630930         0   2.630930   
4    (3, 3)   (10, 3)     1.0  tran3   0.5   2.630930         6   5.402174   
6    (3, 0)   (10, 0)     0.5   scan   0.5   2.630930         0   5.402174   
7   (10, 0)  (-1, -1)     1.0   drop   0.5   5.402174         0   5.402174   
8   (10, 3)  (-1, -1)     1.0   drop   0.5   5.402174         6   5.402174   
5   (26, 2)   (68, 2)     1.0  tran2   0.0   8.925893         4  13.328066   
2   (68, 2)   (68, 0)     0.2   rein   0.0  13.328066         4  13.328066   
9   (68, 2)  (-1, -1)     0.4   drop   0.0  13.328066         4  13.328066   

   target_y  vert  
0       0.0     0  
3       0.0     0  
1       6.0     1  
4       6.0     0  
6       0.0     0  
7       1.0     1  
8

/tmp/ipykernel_54117/3469179521.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  out.loc[idx, 'target_y'] = out.loc[idx, 'source_y'] + 0.5


In [353]:
import numpy as np
import pandas as pd

def assign_vertical_positions(edgelist: pd.DataFrame) -> pd.DataFrame:
    """
    Compute 'bot' and 'top' positions for horizontal edges (vert == 0),
    based on parent edges, sister edges, and flux.
    """

    out = edgelist.copy()
    out['bot'] = np.nan
    out['top'] = np.nan

    horizontal_edges = out[out['vert'] == 0]

    for idx, edge in horizontal_edges.iterrows():
        if edge['state'] == 'load':
            out.at[idx, 'bot'] = edge['source_y']
            out.at[idx, 'top'] = out.at[idx, 'bot'] + out.at[idx, 'flux']
            continue
        parent_edges = get_parent_edges(edge, out)
        num_parents = len(parent_edges)

        if num_parents == 0:
            out.at[idx, 'bot'] = edge['source_y']

        elif num_parents == 1:
            parent_vert = parent_edges['vert'].values[0]

            if parent_vert != 0:
                out.at[idx, 'bot'] = edge['source_y']
            else:
                sister = get_sister_edge(edge, out)
                if len(sister) == 0:
                    out.at[idx, 'bot'] = parent_edges['bot'].values[0]
                    out.at[idx, 'top'] = out.at[idx, 'bot'] + out.at[idx, 'flux']
                    print(edge)
                    continue
                if len(sister) == 1:
                    sister_vert = sister['vert'].values[0]
                    if sister_vert == 1:
                        out.at[idx, 'bot'] = parent_edges['bot'].values[0]
                    elif sister_vert == -1:
                        out.at[idx, 'bot'] = parent_edges['bot'].values[0] + \
                                                parent_edges['flux'].values[0] - \
                                                edge['flux']
                else:
                    out.at[idx, 'bot'] = parent_edges['bot'].values[0] + \
                        parent_edges['flux'].values[0] - \
                        edge['flux'] - sister.loc[sister['vert']==1]['flux']


        elif num_parents == 2:
            nonzero_vert = parent_edges[parent_edges['vert'] != 0]['vert'].values[0]
            zero_vert_parent = parent_edges[parent_edges['vert'] == 0]

            if nonzero_vert == 1:

                out.at[idx, 'bot'] = zero_vert_parent['bot'].values[0] + \
                                        zero_vert_parent['flux'].values[0] - \
                                        edge['flux']
            else:
                out.at[idx, 'bot'] = zero_vert_parent['bot'].values[0]

        # Compute top based on flux
        out.at[idx, 'top'] = out.at[idx, 'bot'] + out.at[idx, 'flux']

    return out


# ----------------------
# Helper functions
# ----------------------




vert_pos = assign_vertical_positions(vertified)
print(vert_pos)

source       (1, 0)
target       (3, 0)
weight          1.0
state          scan
flux            1.0
source_x        1.0
source_y          0
target_x    2.63093
target_y        0.0
vert              0
bot             NaN
top             NaN
Name: 3, dtype: object
     source    target  weight  state  flux   source_x  source_y   target_x  \
0  (-1, -1)    (1, 0)     1.0   load   1.0   0.000000         0   1.000000   
3    (1, 0)    (3, 0)     1.0   scan   1.0   1.000000         0   2.630930   
1    (3, 0)    (3, 3)     0.5   init   0.5   2.630930         0   2.630930   
4    (3, 3)   (10, 3)     1.0  tran3   0.5   2.630930         6   5.402174   
6    (3, 0)   (10, 0)     0.5   scan   0.5   2.630930         0   5.402174   
7   (10, 0)  (-1, -1)     1.0   drop   0.5   5.402174         0   5.402174   
8   (10, 3)  (-1, -1)     1.0   drop   0.5   5.402174         6   5.402174   
5   (26, 2)   (68, 2)     1.0  tran2   0.0   8.925893         4  13.328066   
2   (68, 2)   (68, 0)     0.2   rei

In [354]:

def get_child_edges(edge: pd.Series, edgelist: pd.DataFrame) -> pd.DataFrame:
    """Return edges whose source is the current edge's target."""
    return edgelist[edgelist['source'] == edge['target']]


def assign_vertical_edges(edgelist: pd.DataFrame) -> pd.DataFrame:
    """
    Assign bot/top/left/right positions for vertical edges (vert != 0).
    """

    out = edgelist.copy()
    out[['top_left', 'bot_left']] = np.nan

    vertical_edges = out[out['vert'] != 0]

    for idx, edge in vertical_edges.iterrows():
        half_flux = 0.5 * edge['flux']
        parents = get_parent_edges(edge, out)
        children = get_child_edges(edge, out)

        source_left  = edge['source_x'] - half_flux
        target_left  = edge['target_x'] - half_flux

        # # ---- UPWARD EDGE ----
        if edge['vert'] == 1:
            if parents.empty or children.empty:
                continue

            parent = parents.iloc[0]
            child = children.iloc[0]
            if edge['state'] == 'drop':
                out.at[idx, 'bot'] = parent['top']
                out.at[idx, 'top'] = edge['target_y']
            else:
                out.at[idx, 'bot'] = parent['bot'] + parent['flux']
                out.at[idx, 'top'] = child['bot']
            #corners of the parallelagram
            out.at[idx, 'bot_left']  =  source_left 
            out.at[idx, 'top_left']  =  target_left 

        # ---- DOWNWARD EDGE ----
        elif edge['vert'] == -1:
            if parents.empty or children.empty:
                continue

            parent = parents.iloc[0]
            child = children.iloc[0]
            if edge['state'] == 'drop':
                out.at[idx, 'top'] = parent['bot']
                out.at[idx, 'bot'] = edge['target_y']
            else:
                out.at[idx, 'top'] = parent['bot']
                out.at[idx, 'bot'] = child['bot'] + child['flux']
            
            out.at[idx, 'top_left']  =  source_left 
            out.at[idx, 'bot_left']  =  target_left 


    return out

vert_all = assign_vertical_edges(vert_pos)
print(vert_all)


     source    target  weight  state  flux   source_x  source_y   target_x  \
0  (-1, -1)    (1, 0)     1.0   load   1.0   0.000000         0   1.000000   
3    (1, 0)    (3, 0)     1.0   scan   1.0   1.000000         0   2.630930   
1    (3, 0)    (3, 3)     0.5   init   0.5   2.630930         0   2.630930   
4    (3, 3)   (10, 3)     1.0  tran3   0.5   2.630930         6   5.402174   
6    (3, 0)   (10, 0)     0.5   scan   0.5   2.630930         0   5.402174   
7   (10, 0)  (-1, -1)     1.0   drop   0.5   5.402174         0   5.402174   
8   (10, 3)  (-1, -1)     1.0   drop   0.5   5.402174         6   5.402174   
5   (26, 2)   (68, 2)     1.0  tran2   0.0   8.925893         4  13.328066   
2   (68, 2)   (68, 0)     0.2   rein   0.0  13.328066         4  13.328066   
9   (68, 2)  (-1, -1)     0.4   drop   0.0  13.328066         4  13.328066   

   target_y  vert  bot  top   top_left   bot_left  
0       0.0     0  0.0  1.0        NaN        NaN  
3       0.0     0  0.0  1.0        Na

In [355]:
def adjust_vertical(edgelist: pd.DataFrame):
    out = edgelist.copy()

    equal_mask = out['top_left'].eq(out['bot_left'])
    vert_mask = out['vert'] != 0

    # candidate alignment value
    out["left_value"] = np.select(
        [out["vert"] == 1, out["vert"] == -1],
        [out["bot_left"], out["top_left"]],
        default=np.nan
    )

    # per-source minimum
    out.loc[vert_mask, 'left_value'] = (
        out.loc[vert_mask]
        .groupby('source')['left_value']
        .transform('min')
    )

    # case 1: originally unequal → adjust only one side
    out['top_left'] = np.select(
        [(out['vert'] == -1) & (~equal_mask)],
        [out['left_value']],
        default=out['top_left']
    )

    out['bot_left'] = np.select(
        [(out['vert'] == 1) & (~equal_mask)],
        [out['left_value']],
        default=out['bot_left']
    )

    # case 2: originally equal → move both together
    both_mask = equal_mask & vert_mask
    out.loc[both_mask, 'top_left'] = out.loc[both_mask, 'left_value']
    out.loc[both_mask, 'bot_left'] = out.loc[both_mask, 'left_value']

    # recompute right values
    out['top_right'] = out['top_left'] + out['flux']
    out['bot_right'] = out['bot_left'] + out['flux']

    return out.drop(columns='left_value')



adjusted = adjust_vertical(vert_all)

print(adjusted)

     source    target  weight  state  flux   source_x  source_y   target_x  \
0  (-1, -1)    (1, 0)     1.0   load   1.0   0.000000         0   1.000000   
3    (1, 0)    (3, 0)     1.0   scan   1.0   1.000000         0   2.630930   
1    (3, 0)    (3, 3)     0.5   init   0.5   2.630930         0   2.630930   
4    (3, 3)   (10, 3)     1.0  tran3   0.5   2.630930         6   5.402174   
6    (3, 0)   (10, 0)     0.5   scan   0.5   2.630930         0   5.402174   
7   (10, 0)  (-1, -1)     1.0   drop   0.5   5.402174         0   5.402174   
8   (10, 3)  (-1, -1)     1.0   drop   0.5   5.402174         6   5.402174   
5   (26, 2)   (68, 2)     1.0  tran2   0.0   8.925893         4  13.328066   
2   (68, 2)   (68, 0)     0.2   rein   0.0  13.328066         4  13.328066   
9   (68, 2)  (-1, -1)     0.4   drop   0.0  13.328066         4  13.328066   

   target_y  vert  bot  top   top_left   bot_left  top_right  bot_right  
0       0.0     0  0.0  1.0        NaN        NaN        NaN       

In [356]:
import numpy as np
import pandas as pd

def assign_horizontal_edges(edgelist: pd.DataFrame, color_map: dict) -> pd.DataFrame:
    """
    Assign left/right positions for horizontal edges (vert == 0).
    """

    out = edgelist.copy()
    horizontal = out[out['vert'] == 0]

    for idx, edge in horizontal.iterrows():
        parents = get_parent_edges(edge, out)
        children = get_child_edges(edge, out)

        # ------------------
        # LEFT coordinate
        # ------------------
        if parents.empty:
            left = edge['source_x']

        elif len(parents) == 1:

            left = parents.iloc[0]['bot_right']

        else:
            # Multiple parents: choose vertical parent
            vert_parent = parents[parents['vert'] != 0]
            if not vert_parent.empty:
                if vert_parent['vert'].values[0] == 1:
                    left = vert_parent.iloc[0]['top_right']
                elif vert_parent['vert'].values[0] == -1:
                    left = vert_parent.iloc[0]['bot_right']
            else:
                left = parents.iloc[0]['bot_right']

        # ------------------
        # RIGHT coordinate
        # ------------------
        if not children.empty and (children['vert'] != 0).any():
            vert_child = children[children['vert'] != 0]
            if vert_child.iloc[0]['vert'] == 1:
                right = vert_child.iloc[0]['bot_left']
            else:
                right = vert_child.iloc[0]['top_left']

        else:
            # Look at child's vertical parent
            if not children.empty:
                child = children.iloc[0]
                child_parents = get_parent_edges(child, out)
                vert_child_parent = child_parents[child_parents['vert'] != 0]

                if not vert_child_parent.empty:
                    if vert_child_parent.iloc[0]['vert'] == 1:
                        right = vert_child_parent.iloc[0]['bot_right']
                    else:
                        right = vert_child_parent.iloc[0]['top_right']
                else:
                    right = edge['target_x']
                    print(child)
            else:
                right = np.nan

        if edge['state'] == 'load':
            left = edge['source_x']
        out.loc[idx, ['top_left', 'bot_left']] = left
        out.loc[idx, ['top_right', 'bot_right']] = right

    # ------------------
    # Assign colors
    # ------------------
    out['colour'] = out['state'].map(color_map)


    return out

rects = assign_horizontal_edges(adjusted, color_map=color_map)
print(rects)

source        (1, 0)
target        (3, 0)
weight           1.0
state           scan
flux             1.0
source_x         1.0
source_y           0
target_x     2.63093
target_y         0.0
vert               0
bot              0.0
top              1.0
top_left         NaN
bot_left         NaN
top_right        NaN
bot_right        NaN
Name: 3, dtype: object
     source    target  weight  state  flux   source_x  source_y   target_x  \
0  (-1, -1)    (1, 0)     1.0   load   1.0   0.000000         0   1.000000   
3    (1, 0)    (3, 0)     1.0   scan   1.0   1.000000         0   2.630930   
1    (3, 0)    (3, 3)     0.5   init   0.5   2.630930         0   2.630930   
4    (3, 3)   (10, 3)     1.0  tran3   0.5   2.630930         6   5.402174   
6    (3, 0)   (10, 0)     0.5   scan   0.5   2.630930         0   5.402174   
7   (10, 0)  (-1, -1)     1.0   drop   0.5   5.402174         0   5.402174   
8   (10, 3)  (-1, -1)     1.0   drop   0.5   5.402174         6   5.402174   
5   (26, 2)   (68

In [357]:
import pandas as pd

def rectangles_to_circles(rectangles: pd.DataFrame) -> pd.DataFrame:
    """
    Convert rectangle edges into circles for plotting.
    Each rectangle can generate multiple circles based on corners and child edges.
    """
    circles = []

    for _, rect in rectangles.iterrows():
        # Dictionary of corners
        corners = {
            'top_left': (rect['top_left'], rect['top']),
            'top_right': (rect['top_right'], rect['top']),
            'bot_left': (rect['bot_left'], rect['bot']),
            'bot_right': (rect['bot_right'], rect['bot'])
        }

        # Base circle: pick a primary corner depending on vert
        if rect['vert'] == 1:
            primary_corner = 'bot_left'
            primary_quarter = 1
            secondary_corner = 'top_right'
            secondary_quarter = 3
        elif rect['vert'] == -1:
            primary_corner = 'top_left'
            primary_quarter = 4
            secondary_corner = 'bot_right'
            secondary_quarter = 2
        else:
            continue

        # Main circle
        circles.append({
            'centre_x': corners[primary_corner][0],
            'centre_y': corners[primary_corner][1],
            'radius': rect['flux'],
            'quarter': primary_quarter,
            'colour': rect['colour']
        })

        # If rectangle has children, add secondary circle
        children = get_child_edges(rect, rectangles)
        if len(children) > 0:
            child_colour = children['colour'].values[0]

            circles.append({
                'centre_x': corners[secondary_corner][0],
                'centre_y': corners[secondary_corner][1],
                'radius': rect['flux'],
                'quarter': secondary_quarter,
                'colour': child_colour
            })

    return pd.DataFrame(circles).dropna()

circles = rectangles_to_circles(rectangles=rects)
print(circles)


    centre_x  centre_y  radius  quarter    colour
0   2.380930       1.0     0.5        1     green
1   2.880930       6.0     0.5        3  darkblue
2   5.152174       0.5     0.5        1    purple
3   5.652174       1.0     0.5        3    purple
4   5.152174       6.5     0.5        1    purple
5   5.652174       7.0     0.5        3    purple
7  13.328066       4.0     0.0        1    purple
8  13.328066       5.0     0.0        3    purple


In [358]:
import cairo

# -----------------------
# Data bounds
# -----------------------
x_min = min(rects['top_left'].min(), circles['centre_x'].min())
x_max = max(rects['top_right'].max(), circles['centre_x'].max())
y_min = min(rects['bot'].min(), circles['centre_y'].min())
y_max = max(rects['top'].max(), circles['centre_y'].max())

# -----------------------
# Canvas parameters
# -----------------------
px_per_unit = 100  # pixels per 1 data unit

WIDTH = int((x_max - x_min) * px_per_unit)
HEIGHT = int((y_max - y_min) * px_per_unit)

# Scale factors
x_scale = WIDTH / (x_max - x_min)
y_scale = HEIGHT / (y_max - y_min)
scale = min(x_scale, y_scale)  # uniform scale to preserve aspect ratio

# Margins to center the drawing
x_margin = (WIDTH - scale * (x_max - x_min)) / 2
y_margin = (HEIGHT - scale * (y_max - y_min)) / 2

# -----------------------
# Coordinate transforms
# -----------------------
def sx(x):
    """Transform data x-coordinate to pixel x-coordinate."""
    return x_margin + (x - x_min) * scale

def sy(y):
    """Transform data y-coordinate to pixel y-coordinate (y-axis inverted)."""
    return HEIGHT - (y_margin + (y - y_min) * scale)


In [359]:
surface = cairo.SVGSurface('graph.svg', WIDTH, HEIGHT)
ctx = cairo.Context(surface)

ctx.set_source_rgb(1, 1, 1)
ctx.paint()


In [360]:
for _, r in rects.iterrows():
    # Coordinates in pixels
    x0, y0 = sx(r.top_left), sy(r.top)
    x1, y1 = sx(r.top_right), sy(r.top)
    x2, y2 = sx(r.bot_right), sy(r.bot)
    x3, y3 = sx(r.bot_left), sy(r.bot)

    # Draw as a polygon (handles any rectangle orientation)
    ctx.move_to(x0, y0)
    ctx.line_to(x1, y1)
    ctx.line_to(x2, y2)
    ctx.line_to(x3, y3)
    ctx.close_path()

    # Fill with color and alpha
    print(r.state)
    ctx.set_source_rgba(*COLOURS[r.colour], 0.7)
    ctx.fill()

    # Optional stroke
    # ctx.set_source_rgb(0, 0, 0)
    # ctx.stroke()


load
scan
init
tran3
scan
drop
drop
tran2
rein
drop


In [361]:
QUARTER_ANGLES = {
    1: (0, math.pi/2),
    2: (math.pi/2, math.pi),
    3: (math.pi, 3*math.pi/2),
    4: (3*math.pi/2, 2*math.pi),
}

for _, a in circles.iterrows():
    cx = sx(a.centre_x)
    cy = sy(a.centre_y)
    r  = sx(a.centre_x + a.radius) - sx(a.centre_x)

    start, end = QUARTER_ANGLES[a.quarter]

    ctx.move_to(cx, cy)
    ctx.arc(cx, cy, r, start, end)
    ctx.close_path()

    ctx.set_source_rgba(*COLOURS[a.colour], 0.7)
    ctx.fill()
    # ctx.set_source_rgb(0, 0, 0)
    # ctx.stroke()


In [362]:
surface.finish()